## A. Thu thập dữ liệu

### Ngữ cảnh và động lực 

### Nguồn dữ liệu

- Tập dữ liệu này được sử dụng cho việc phân tích trong lĩnh vực chuỗi cung ứng bởi công ty DataCo Global.
- Tập dữ liệu gồm:
  - Dữ liệu có cấu trúc: DataCoSupplyChainDataset.csv.
  - Dữ liệu không có cấu trúc: tokenized_access_logs.csv.
  - Ngoài ra có đính kèm một file DescriptionDataCoSupplyChain.csv để mô tả các biến trong file DataCoSupplyChainDataset.csv.
- Tập dữ liệu được thu thập từ trang Kaggle: https://www.kaggle.com/datasets/shashwatwork/dataco-smart-supply-chain-for-big-data-analysis/data.
- Trích dẫn: Constante, Fabian; Silva, Fernando; Pereira, António (2019), “DataCo SMART SUPPLY CHAIN FOR BIG DATA ANALYSIS”, Mendeley Data, V5, doi: 10.17632/8gx2fvg2k6.5
- License của dataset: CC BY 4.0

## B. Khám phá dữ liệu

### Import thư viện

In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt


from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from langdetect import detect
from wordcloud import WordCloud
#!pip install nltk
import nltk
#nltk.download('stopwords')

### Đọc dữ liệu

In [2]:
current_directory = os.getcwd()
relative_path = "../datasets/DataCoSupplyChainDataset.csv"

file_path = os.path.join(current_directory, relative_path)
raw_df = pd.read_csv(file_path, encoding='latin1')
raw_df.head(5)

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,Customer City,...,Order Zipcode,Product Card Id,Product Category Id,Product Description,Product Image,Product Name,Product Price,Product Status,shipping date (DateOrders),Shipping Mode
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,2/3/2018 22:56,Standard Class
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/18/2018 12:27,Standard Class
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,73,Sporting Goods,San Jose,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/17/2018 12:06,Standard Class
3,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,73,Sporting Goods,Los Angeles,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/16/2018 11:45,Standard Class
4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,73,Sporting Goods,Caguas,...,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/15/2018 11:24,Standard Class


### Tập dữ liệu có bao nhiêu dòng, cột?

In [3]:
shape = raw_df.shape
print(f'Có {shape[0]} dòng và {shape[1]} cột')

Có 180519 dòng và 53 cột


### Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

- Mỗi dòng chứa thông tin chi tiết về một giao dịch mua hàng cụ thể hoặc về một sản phẩm được mua trong đó. Bao gồm các chi tiết như thông tin khách hàng, thông tin đơn hàng, chuyển phát đơn hàng,...
- Nếu các dòng có ý nghĩa khác nhau sẽ làm cho tính tin cậy của tập dữ liệu bị giảm đi và có thể dẫn đến một vài sự nhầm lẫn và tính nhất quán giữa chúng.

### Mỗi cột có ý nghĩa gì?


| Tên cột                        | Mô tả                                       |
|--------------------------------|----------------------------------------------|
| Type                           | Loại giao dịch                              |
| Days for shipping (real)       | Số ngày thực tế để vận chuyển sản phẩm      |
| Days for shipment (scheduled)  | Số ngày dự kiến giao hàng của sản phẩm      |
| Benefit per order              | Lợi nhuận trên mỗi đơn đặt hàng             |
| Sales per customer             | Tổng doanh số trên mỗi khách hàng           |
| Delivery Status                | Trạng thái giao hàng: Giao hàng nhanh, Giao hàng trễ, Huỷ giao hàng, Giao hàng đúng hẹn |
| Late_delivery_risk             | Biến phân loại chỉ ra liệu việc gửi hàng có trễ hay không (1: có trễ, 0: không trễ) |
| Category Id                    | Mã danh mục sản phẩm                        |
| Category Name                  | Mô tả về danh mục sản phẩm                  |
| Customer City                  | Thành phố mà khách hàng thực hiện mua hàng  |
| Customer Country               | Quốc gia mà khách hàng thực hiện mua hàng   |
| Customer Email                 | Email của khách hàng                        |
| Customer Fname                 | Tên của khách hàng                          |
| Customer Id                    | Mã khách hàng                               |
| Customer Lname                 | Họ của khách hàng                           |
| Customer Password              | Mã khóa của khách hàng được che giấu        |
| Customer Segment               | Loại khách hàng: Tiêu dùng, Doanh nghiệp, Văn phòng tại nhà |
| Customer State                 | Bang mà cửa hàng mua hàng của khách hàng thuộc về |
| Customer Street                | Đường mà cửa hàng mua hàng của khách hàng thuộc về |
| Customer Zipcode               | Mã bưu chính của khách hàng                |
| Department Id                  | Mã phòng ban của cửa hàng                   |
| Department Name                | Tên phòng ban của cửa hàng                  |
| Latitude                       | Vĩ độ tương ứng với vị trí của cửa hàng     |
| Longitude                      | Kinh độ tương ứng với vị trí của cửa hàng   |
| Market                         | Thị trường mà đơn hàng được giao: Châu Phi, Châu Âu, LATAM, Châu Á Thái Bình Dương, USCA |
| Order City                     | Thành phố đích của đơn hàng                |
| Order Country                  | Quốc gia đích của đơn hàng                 |
| Order Customer Id              | Mã đơn đặt hàng của khách hàng             |
| order date (DateOrders)        | Ngày mà đơn hàng được tạo                  |
| Order Id                       | Mã đơn hàng                                 |
| Order Item Cardprod Id         | Mã sản phẩm được tạo thông qua đầu đọc RFID |
| Order Item Discount            | Giá trị giảm giá mục đơn hàng             |
| Order Item Discount Rate       | Tỷ lệ giảm giá mục đơn hàng               |
| Order Item Id                  | Mã mục đơn hàng                            |
| Order Item Product Price       | Giá của sản phẩm không có giảm giá          |
| Order Item Profit Ratio        | Tỷ lệ lợi nhuận của mục đơn hàng          |
| Order Item Quantity            | Số lượng sản phẩm trên mỗi đơn hàng        |
| Sales                          | Giá trị trong doanh số                      |
| Order Item Total               | Tổng số tiền trên mỗi đơn hàng             |
| Order Profit Per Order         | Lợi nhuận trên mỗi đơn hàng                |
| Order Region                   | Vùng trên thế giới mà đơn hàng được giao: Đông Nam Á, Nam Á, Ô xê nia, Đông Á, Tây Á, Tây bắc Hoa Kỳ, Trung tâm Hoa Kỳ, Tây Phi, Trung Phi, Bắc Phi, Tây Âu, Bắc Âu, Caribbe, Nam Mỹ, Đông Phi, Nam Âu, Đông bắc Hoa Kỳ, Canada, Nam Phi, Trung Á, Châu Âu, Trung Mỹ, Đông Âu, Nam của Hoa Kỳ |
| Order State                    | Tiểu bang của khu vực mà đơn hàng được giao |
| Order Status                   | Tình trạng đơn hàng: HOÀN THÀNH, ĐANG CHỜ, ĐÓNG, ĐANG CHỜ THANH TOÁN, ĐÃ HUỶ, ĐANG XỬ LÝ, NHẬN DẤU, ĐANG GIỮ, ĐÁNH GIÁ THANH TOÁN |
| Product Card Id                | Mã sản phẩm                                 |
| Product Category Id            | Mã danh mục sản phẩm                        |
| Product Description            | Mô tả về sản phẩm                          |
| Product Image                  | Liên kết đến việc xem và mua sản phẩm       |
| Product Name                   | Tên sản phẩm                                |
| Product Price                  | Giá sản phẩm                                |
| Product Status                 | Tình trạng kho hàng: Nếu là 1 thì không có sẵn, 0 thì sản phẩm có sẵn |
| Shipping date (DateOrders)     | Ngày và giờ chính xác của lô hàng           |
| Shipping Mode                  | Các chế độ vận chuyển sau được trình bày: Lớp tiêu chuẩn, Lớp đầu tiên, Lớp thứ hai, Cùng ngày |


### Liệu có các dòng nào bị trùng lặp không?

In [4]:
duplicate_rows = raw_df[raw_df.duplicated()]
duplicate_rows.shape[0]

0

Tập dữ liệu này không có dòng nào bị trùng lặp!

### Mỗi cột hiện đang có kiểu dữ liệu gì?

In [5]:
raw_df.dtypes

Type                              object
Days for shipping (real)           int64
Days for shipment (scheduled)      int64
Benefit per order                float64
Sales per customer               float64
Delivery Status                   object
Late_delivery_risk                 int64
Category Id                        int64
Category Name                     object
Customer City                     object
Customer Country                  object
Customer Email                    object
Customer Fname                    object
Customer Id                        int64
Customer Lname                    object
Customer Password                 object
Customer Segment                  object
Customer State                    object
Customer Street                   object
Customer Zipcode                 float64
Department Id                      int64
Department Name                   object
Latitude                         float64
Longitude                        float64
Market          

Kết hợp với quan sát dữ liệu trên DataCoSupplyChainDataset.csv, các cột đều có kiểu dữ liệu phù hợp. Ngoại trừ 2 cột là `order date (DateOrders)` và `shipping date (DateOrders)` cần phải chuyển về kiểu `datetime`.

In [6]:
cols = ['order date (DateOrders)', 'shipping date (DateOrders)']
raw_df[cols] = raw_df[cols].apply(pd.to_datetime)
raw_df.dtypes

Type                                     object
Days for shipping (real)                  int64
Days for shipment (scheduled)             int64
Benefit per order                       float64
Sales per customer                      float64
Delivery Status                          object
Late_delivery_risk                        int64
Category Id                               int64
Category Name                            object
Customer City                            object
Customer Country                         object
Customer Email                           object
Customer Fname                           object
Customer Id                               int64
Customer Lname                           object
Customer Password                        object
Customer Segment                         object
Customer State                           object
Customer Street                          object
Customer Zipcode                        float64
Department Id                           

### Với mỗi cột có kiểu là dữ liệu số, những giá trị trong cột đó được phân bố như thế nào?

In [7]:
num_col_df = raw_df.select_dtypes(include='number')
num_col_df.head(5)

,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Late_delivery_risk,Category Id,Customer Id,Customer Zipcode,Department Id,Latitude,...,Order Item Quantity,Sales,Order Item Total,Order Profit Per Order,Order Zipcode,Product Card Id,Product Category Id,Product Description,Product Price,Product Status
0,3,4,91.250000,314.640015,0,73,20755,725.0,2,18.251453,...,1,327.75,314.640015,91.250000,NaN,1360,73,NaN,327.75,0
1,5,4,-249.089996,311.359985,1,73,19492,725.0,2,18.279451,...,1,327.75,311.359985,-249.089996,NaN,1360,73,NaN,327.75,0
2,4,4,-247.779999,309.720001,0,73,19491,95125.0,2,37.292233,...,1,327.75,309.720001,-247.779999,NaN,1360,73,NaN,327.75,0
3,3,4,22.860001,304.809998,0,73,19490,90027.0,2,34.125946,...,1,327.75,304.809998,22.860001,NaN,1360,73,NaN,327.75,0
4,2,4,134.210007,298.250000,0,73,19489,725.0,2,18.253769,...,1,327.75,298.250000,134.210007,NaN,1360,73,NaN,327.75,0


Với tất cả các cột là dữ liệu số trên, ta cần phải tính các giá trị sau cho từng cột:
- `missing_ratio`: tỉ lệ (0-100%) các giá trị bị thiếu.
- `min`: giá trị nhỏ nhất trong cột đó.
- `lower_quartile`: giá trị tứ phân vị thứ nhất của cột.
- `median`: giá trị trung vị của cột.
- `upper_quartile`: giá trị tứ phân vị thứ ba của cột.
- `max`: giá trị lớn nhất của cột.

In [8]:
def missing_ratio(s):
    missing_count = s.isnull().sum()
    total_values = len(s)
    ratio = (missing_count / total_values) * 100
    return ratio.round(1)

def median(df):
    return df.median().round(1)

def lower_quartile(df):
    return df.quantile(0.25).round(1)

def upper_quartile(df):
    return df.quantile(0.75).round(1)

Dùng phương thức `.agg()` để áp dụng cho tất cả các cột của `numerical_cols`, mỗi dòng sẽ là giá trị cần tính đã được định nghĩa ở trên.

In [9]:
num_col_info_df = num_col_df.agg([missing_ratio, "min", lower_quartile, median, upper_quartile, "max"])
num_col_info_df

,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Late_delivery_risk,Category Id,Customer Id,Customer Zipcode,Department Id,Latitude,...,Order Item Quantity,Sales,Order Item Total,Order Profit Per Order,Order Zipcode,Product Card Id,Product Category Id,Product Description,Product Price,Product Status
missing_ratio,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.00000,0.00000,0.000000,86.2,0.0,0.0,100.0,0.00000,0.0
min,0.0,0.0,-4274.979980,7.49000,0.0,2.0,1.0,603.0,2.0,-33.937553,...,1.0,9.99000,7.49000,-4274.979980,1040.0,19.0,2.0,NaN,9.99000,0.0
lower_quartile,2.0,2.0,7.000000,104.40000,0.0,18.0,3258.5,725.0,4.0,18.300000,...,1.0,120.00000,104.40000,7.000000,23464.0,403.0,18.0,NaN,50.00000,0.0
median,3.0,4.0,31.500000,164.00000,1.0,29.0,6457.0,19380.0,5.0,33.100000,...,1.0,199.90000,164.00000,31.500000,59405.0,627.0,29.0,NaN,60.00000,0.0
upper_quartile,5.0,4.0,64.800000,247.40000,1.0,45.0,9779.0,78207.0,7.0,39.300000,...,3.0,300.00000,247.40000,64.800000,90008.0,1004.0,45.0,NaN,200.00000,0.0
max,6.0,4.0,911.799988,1939.98999,1.0,76.0,20757.0,99205.0,12.0,48.781933,...,5.0,1999.98999,1939.98999,911.799988,99301.0,1363.0,76.0,NaN,1999.98999,0.0


Từ num_col_info_df, thấy rằng có vài cột có missing_ratio rất lớn, nên nhóm sẽ loại bỏ những cột có missing_ratio > 50%

In [10]:
large_missing_raio_cols = num_col_info_df.columns[num_col_info_df.iloc[0] >= 50.0].tolist()
print(large_missing_raio_cols)

#drop
num_col_df = num_col_df.drop(large_missing_raio_cols,axis=1)
raw_df = raw_df.drop(large_missing_raio_cols,axis=1)
raw_df.shape

['Order Zipcode', 'Product Description']


(180519, 51)

Kiểm tra xem cột nào có missing_ratio > 0 để tiến hành fill các giá trị bị mất

In [11]:
num_col_info_df = num_col_df.agg([missing_ratio, "min", lower_quartile, median, upper_quartile, "max"])
temp = num_col_info_df.columns[num_col_info_df.iloc[0] > 0.0].tolist()
temp

[]

Không có cột nào trong `numerical_cols` bị mất mát dữ liệu ngoài 2 cột đã bị drop ở trên.

### Với mỗi cột có kiểu là dạng phân loại, những giá trị trong cột đó được phân bố như thế nào?

In [12]:
cate_col_df = raw_df.select_dtypes(include=['object'])
cate_col_df.head()

,Type,Delivery Status,Category Name,Customer City,Customer Country,Customer Email,Customer Fname,Customer Lname,Customer Password,Customer Segment,...,Department Name,Market,Order City,Order Country,Order Region,Order State,Order Status,Product Image,Product Name,Shipping Mode
0,DEBIT,Advance shipping,Sporting Goods,Caguas,Puerto Rico,XXXXXXXXX,Cally,Holloway,XXXXXXXXX,Consumer,...,Fitness,Pacific Asia,Bekasi,Indonesia,Southeast Asia,Java Occidental,COMPLETE,http://images.acmesports.sports/Smart+watch,Smart watch,Standard Class
1,TRANSFER,Late delivery,Sporting Goods,Caguas,Puerto Rico,XXXXXXXXX,Irene,Luna,XXXXXXXXX,Consumer,...,Fitness,Pacific Asia,Bikaner,India,South Asia,Rajastán,PENDING,http://images.acmesports.sports/Smart+watch,Smart watch,Standard Class
2,CASH,Shipping on time,Sporting Goods,San Jose,EE. UU.,XXXXXXXXX,Gillian,Maldonado,XXXXXXXXX,Consumer,...,Fitness,Pacific Asia,Bikaner,India,South Asia,Rajastán,CLOSED,http://images.acmesports.sports/Smart+watch,Smart watch,Standard Class
3,DEBIT,Advance shipping,Sporting Goods,Los Angeles,EE. UU.,XXXXXXXXX,Tana,Tate,XXXXXXXXX,Home Office,...,Fitness,Pacific Asia,Townsville,Australia,Oceania,Queensland,COMPLETE,http://images.acmesports.sports/Smart+watch,Smart watch,Standard Class
4,PAYMENT,Advance shipping,Sporting Goods,Caguas,Puerto Rico,XXXXXXXXX,Orli,Hendricks,XXXXXXXXX,Corporate,...,Fitness,Pacific Asia,Townsville,Australia,Oceania,Queensland,PENDING_PAYMENT,http://images.acmesports.sports/Smart+watch,Smart watch,Standard Class


Đối với các cột có kiểu là dạng phân loại, ta cũng sẽ tính:
- `missing_ratio`: tỉ lệ (0-100%) các giá trị bị thiếu.
- `num_values`: số lượng các giá trị độc nhất(unique) trong cột.
- `value_ratios`: tỉ lệ % tổng từng giá trị unique trên số lượng giá trị non-null của cột.

In [13]:
def missing_ratio(s):
    missing_count = s.isnull().sum()
    total_values = len(s)
    ratio = (missing_count / total_values) * 100
    return ratio.round(1)

def num_values(s):
    unique_count = s.nunique()
    return int(unique_count)

def value_ratios(s):
    distribution = s.value_counts().to_dict()
    for key in distribution:
        distribution[key] = (distribution[key]*100 / s.count()).round(1)
    return distribution

cat_col_info_df = cate_col_df.agg([missing_ratio, num_values, value_ratios])
cat_col_info_df

,Type,Delivery Status,Category Name,Customer City,Customer Country,Customer Email,Customer Fname,Customer Lname,Customer Password,Customer Segment,...,Department Name,Market,Order City,Order Country,Order Region,Order State,Order Status,Product Image,Product Name,Shipping Mode
missing_ratio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
num_values,4,4,50,563,2,1,782,1109,1,3,...,11,5,3597,164,23,1089,9,118,118,4
value_ratios,"{'DEBIT': 38.4, 'TRANSFER': 27.6, 'PAYMENT': 2...","{'Late delivery': 54.8, 'Advance shipping': 23...","{'Cleats': 13.6, 'Men's Footwear': 12.3, 'Wome...","{'Caguas': 37.0, 'Chicago': 2.2, 'Los Angeles'...","{'EE. UU.': 61.6, 'Puerto Rico': 38.4}",{'XXXXXXXXX': 100.0},"{'Mary': 36.1, 'James': 1.0, 'Robert': 1.0, 'M...","{'Smith': 35.5, 'Johnson': 0.5, 'Brown': 0.5, ...",{'XXXXXXXXX': 100.0},"{'Consumer': 51.8, 'Corporate': 30.4, 'Home Of...",...,"{'Fan Shop': 37.0, 'Apparel': 27.1, 'Golf': 18...","{'LATAM': 28.6, 'Europe': 27.8, 'Pacific Asia'...","{'Santo Domingo': 1.2, 'New York City': 1.2, '...","{'Estados Unidos': 13.8, 'Francia': 7.3, 'Méxi...","{'Central America': 15.7, 'Western Europe': 15...","{'Inglaterra': 3.7, 'California': 2.8, 'Isla d...","{'COMPLETE': 33.0, 'PENDING_PAYMENT': 22.1, 'P...",{'http://images.acmesports.sports/Perfect+Fitn...,"{'Perfect Fitness Perfect Rip Deck': 13.6, 'Ni...","{'Standard Class': 59.7, 'Second Class': 19.5,..."


In [14]:
missing_raio_cols = cat_col_info_df.columns[cat_col_info_df.iloc[0] > 0.0].tolist()
print(missing_raio_cols)

[]


**Nhận xét**:
- Như đoạn code trên cho thấy rằng, ở những cột categorical này không có cột nào bị mất mát dữ liệu. Tuy nhiên, có 2 cột là `Customer Email` và `Customer Password`, có lẽ vì lí do bảo mật nên dữ liệu của 2 cột này bị thay thế thành giá trị ẩn `'XXXXXXXXX'`.
- Hơn nữa, tỉ lệ value_ratios của giá trị này ở 2 cột trên đều là 100% nên nhóm xem chúng là missing values, không có giá trị gì trong việc phân tích dữ liệu sau này.
- Do đó, 2 cột này sẽ bị xóa khỏi dataframe.

In [15]:
cate_col_df = cate_col_df.drop(['Customer Email','Customer Password'],axis=1)
raw_df = raw_df.drop(['Customer Email','Customer Password'],axis=1)
raw_df.shape

(180519, 49)